## Probability of LoF tolerance for a set of enhancers 

In [8]:
# Khurana paper: 
# https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1008663

CONSTRAINT_TOOLS_DATA = '/scratch/ucgd/lustre-work/quinlan/data-shared/constraint-tools'

import pandas as pd

# Read in the Excel file, which contains multiple sheets
khurana_table_S2 = pd.read_excel(f'{CONSTRAINT_TOOLS_DATA}/khurana/table-S2.xlsx', sheet_name=None)

In [11]:
khurana_table_S2.keys()

dict_keys(['SupplementaryTable3', 'LoF-tolerant enhancers', 'Low-LoF-tolerance enhancers', 'Feature importance'])

In [13]:
enhancers_and_khurana_scores = khurana_table_S2['SupplementaryTable3']
enhancers_and_khurana_scores

,enhancer,original,predict LoF-tolerant,prob LoF-tolerance
0,chr13-101408800-101409200,NotDeleted,Low-LoF-tolerance,0.248020
1,chr18-45880000-45880200,NotDeleted,LoF-tolerant,0.646635
2,chr5-174837800-174840000,NotDeleted,LoF-tolerant,0.881760
3,chr9-115430200-115432000,NotDeleted,LoF-tolerant,0.903623
4,chr11-134077600-134078200,NotDeleted,LoF-tolerant,0.729862
...,...,...,...,...
245088,chr22-51047400-51049200,NotDeleted,LoF-tolerant,0.592781
245089,chr5-55940400-55941400,NotDeleted,LoF-tolerant,0.647929
245090,chr10-61223600-61224000,NotDeleted,LoF-tolerant,0.516712
245091,chr14-63648400-63648600,NotDeleted,LoF-tolerant,0.665555


In [ ]:
# TODO 
# 1. assign chen scores to the enhancers listed above (see aps.*) (did Khurana use hg19 or hg38?)
# 2. Scatter plot zscores and Khurana scores 

In [ ]:
# TODO
# 1. pull out "vista" and "LoF tolerant" enhancers from "khurana_table_S2"
# 2. annotate all enhancers as "vista" or "LoF tolerant" or "neither"
# 3. Look at the CDF of scores (Chen, McHale, Khurana) for all enhancers, just those annotated as "vista", and just those annotated as "LoF-tolerant", similar to Fig 4A

In [ ]:
# TODO [LONG SHOT] 
# Do SNVs (het or hom alt) in vista enhancers avoid the motifs that are enriched in those enhancers? 